In [1]:
from promise import Promise
from promise.dataloader import DataLoader

In [2]:
class UserLoader(DataLoader):
    cache = False
    
    def batch_load_fn(self, keys):
        print('SELECT * FROM tbl_user WHERE id IN {}'.format(keys))
        return Promise.resolve([
            {'id': id, 'friend_ids': [i + id for i in range(10, 13)]} 
            for id in keys
        ])

user_loader = UserLoader()

In [3]:
user_promise = user_loader.load(1)
print(user_promise)

SELECT * FROM tbl_user WHERE id IN [1]
<Promise at 0x1031747b8 fulfilled with {'id': 1, 'friend_ids': [11, 12, 13]}>


In [8]:
user_promise.then(lambda user: user_loader.load_many(user['friend_ids']))
# user_promise.then(lambda user: user_loader.load_many(user['friend_ids'])).then(
#     lambda users: [
#         user_loader.load_many(user['friend_ids'])
#         for user in users
#     ]
# )


SELECT * FROM tbl_user WHERE id IN [11, 12, 13]
SELECT * FROM tbl_user WHERE id IN [21, 22, 23, 22, 23, 24, 23, 24, 25]


<Promise at 0x1031a7278 fulfilled with [<Promise at 0x1031a7208 fulfilled with [{'id': 21, 'friend_ids': [31, 32, 33]}, {'id': 22, 'friend_ids': [32, 33, 34]}, {'id': 23, 'friend_ids': [33, 34, 35]}]>, <Promise at 0x102ff57f0 fulfilled with [{'id': 22, 'friend_ids': [32, 33, 34]}, {'id': 23, 'friend_ids': [33, 34, 35]}, {'id': 24, 'friend_ids': [34, 35, 36]}]>, <Promise at 0x102ff5860 fulfilled with [{'id': 23, 'friend_ids': [33, 34, 35]}, {'id': 24, 'friend_ids': [34, 35, 36]}, {'id': 25, 'friend_ids': [35, 36, 37]}]>]>

In [5]:
def lam(friend_ids):
    print('start', friend_ids)
    r = user_loader.load_many(friend_ids)
    print('end')
    return r
lam([30, 31, 32])

start [30, 31, 32]
SELECT * FROM tbl_user WHERE id IN [30]
SELECT * FROM tbl_user WHERE id IN [31]
SELECT * FROM tbl_user WHERE id IN [32]
end


<Promise at 0x1031743c8 fulfilled with [{'id': 30, 'friend_ids': [40, 41, 42]}, {'id': 31, 'friend_ids': [41, 42, 43]}, {'id': 32, 'friend_ids': [42, 43, 44]}]>

In [6]:
async def f():
    r = await user_loader.load_many([30, 31, 32])
    print(r)
    return r
import asyncio
loop = asyncio.new_event_loop()
result = loop.run_until_complete(f())
loop.close()

SELECT * FROM tbl_user WHERE id IN [30]
SELECT * FROM tbl_user WHERE id IN [31]
SELECT * FROM tbl_user WHERE id IN [32]
[{'id': 30, 'friend_ids': [40, 41, 42]}, {'id': 31, 'friend_ids': [41, 42, 43]}, {'id': 32, 'friend_ids': [42, 43, 44]}]
